In [23]:
from bs4 import BeautifulSoup
import os

In [24]:
filepath1 = "C:/Users/rschl/Desktop/Image Labeling/annotated data/AmanitaPhalloides/AmanitaPhalloides_1.xml"
filepath2 = "C:/Users/rschl/Desktop/Image Labeling/annotated data/AmanitaPhalloides/AmanitaPhalloides_0.xml"

In [25]:
def parseXML(filepath):
    infile = open(filepath,'r')
    content = infile.read()
    soup = BeautifulSoup(content,'xml')
    width = int(soup.find('width').text)
    height = int(soup.find('height').text)
    xmins = soup.findAll('xmin')
    ymins = soup.findAll('ymin')
    xmax = soup.findAll('xmax')
    ymax = soup.findAll('ymax')
    boxes = []
    for i in range(len(xmins)):
        box = []
        box.append(int(xmins[i].contents[0]))
        box.append(int(ymins[i].contents[0]))
        box.append(int(xmax[i].contents[0]))
        box.append(int(ymax[i].contents[0]))
        boxes.append(box)

    return width, height, boxes


In [26]:
def convert_coord(pascal):
    width = pascal[0]
    height = pascal[1]
    
    list_boxes = pascal[2]

    yolo = []

    for i in range(len(list_boxes)):
        yolo_box = []
        
        xmin = list_boxes[i][0]
        ymin = list_boxes[i][1]
        xmax = list_boxes[i][2]
        ymax = list_boxes[i][3]

        yolo_x = ((xmax-xmin)/2 + xmin)/width
        yolo_y = ((ymax-ymin)/2 + ymin)/height

        yolo_width = round((xmax-xmin)/width,6)
        yolo_height = round((ymax-ymin)/height,6)

        yolo_box.append(format(yolo_x,'.6f'))
        yolo_box.append(format(yolo_y,'.6f'))
        yolo_box.append(format(yolo_width,'.6f'))
        yolo_box.append(format(yolo_height,'.6f'))

        yolo.append(yolo_box)

    return yolo


In [27]:
def convert_yolo_list(yolo_box):
    yolo_complete = ""
    for i in range(len(yolo_box)):    
        curr_box = yolo_box[i]
        yolo_str = "0 "
        for i in range(len(curr_box)):
            yolo_str += curr_box[i]
            yolo_str += " "
        
        yolo_str = yolo_str[:-1]
        yolo_str += "\n"
        yolo_complete += yolo_str


    return yolo_complete

In [28]:
data = [
    'annotated_data\Amanita Pantherina',
    'annotated_data\AmanitaPantherina',
    'annotated_data\AmanitaPhalloides'
    'annotated_data\Conocybe'
    'annotated_data\CoprinopsisAtramentaria'
]

In [29]:
def combined_conversion(file, destination):
    try:
        f = open(destination,'w')
    except:
        print("Something went wrong opening the destination folder")
    try:
        f.write(convert_yolo_list(convert_coord(parseXML(file))))
    except:
        print("Something went wrong writing the text file")

In [30]:
'''
Pass in the folder path of the xml files to be converted
'''

def convert_directory(dir):
    file_list = os.listdir(dir)

    if not os.path.exists(dir + '\yolo'):
        os.makedirs(dir + '\yolo')

    for i in range(len(file_list)-1):
        xml_path = dir + '\\' + file_list[i]
        if '.xml' in xml_path:
            destination = dir + '\yolo' + '\\' + file_list[i][:-3] + "txt"
            combined_conversion(xml_path,destination)


        

In [32]:
convert_directory()